In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import math

dataset = 'cifar10'
train_size = 50000
image_size = 32
batch_size = 32
reduction_ratio = 16
num_classes = 10
momentum = 0.8
initial_learning_rate = 0.001
drop_rate = 0.1
epochs_per_drop = 30.0
steps_per_epoch = train_size // batch_size
epochs = 100

ds_train = tfds.load(dataset, split='train')
ds_test = tfds.load(dataset, split='test')

def normalize_image(ele):
    image = ele['image']
    label = ele['label']
    paddings = tf.constant([[4,4], [4,4], [0,0]])
  
    image = tf.pad(image, paddings, 'CONSTANT')
    image = tf.image.random_crop(image, size = [32,32,3])
    
    image = (image - tf.reduce_min(image))/(tf.reduce_max(image)-tf.reduce_min(image))
    return image, label

ds_train = ds_train.map(normalize_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).repeat().shuffle(100).batch(batch_size).prefetch(1)
ds_test = ds_test.map(normalize_image,  num_parallel_calls=tf.data.experimental.AUTOTUNE).repeat().shuffle(100).batch(batch_size).prefetch(1)

def conv2D(x, filters, kernel=(3,3), strides=(1,1), activation=tf.nn.relu, use_bias=False):
    x = tf.keras.layers.Conv2D(filters, kernel_size=kernel, strides=strides, use_bias=use_bias, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    if activation is not None:
        x = tf.keras.layers.Activation(activation)(x)
    return x

def SE_layer(x, ratio=reduction_ratio):
    depth = list(x.get_shape())[-1]
    prev = x
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(depth//ratio, activation=tf.nn.relu)(x)
    x = tf.keras.layers.Dense(depth, activation=tf.nn.sigmoid)(x)
    x = tf.reshape(x, [-1, 1, 1, depth])
    x = prev * x
    return x

def block(x, trips=[64, 64, 256], first=False, SE=False):
    if first:
        x = conv2D(x, trips[0], kernel=(1,1), strides=(2,2))
    else:
        x = conv2D(x, trips[0], kernel=(1,1))
    x = conv2D(x, trips[1], kernel=(3,3))
    x = conv2D(x, trips[2], kernel=(1,1))
    if SE:
        x = SE_layer(x)
    return x

def meta_block(x, trips=[64, 64, 256], repeats=3, downsample=True, SE=False):
    prev = x
    for r in range(repeats):
        if r == 0 and downsample:
            prev = conv2D(prev, trips[2], kernel=(1,1), strides=(2,2), activation=None)
            x = block(x, trips=trips, first=True, SE=SE)
            x = tf.keras.layers.Add()([x, prev])
        elif r==0 and not downsample:
            prev = conv2D(prev, trips[2], kernel=(1,1), activation=None)
            x = block(x, trips=trips, SE=SE)
            x = tf.keras.layers.Add()([x, prev])
        else:
            prev = x
            x = block(x, trips=trips, SE=SE)
            x = tf.keras.layers.Add()([x, prev])
    return x

inputs = tf.keras.layers.Input(shape=(image_size, image_size, 3))
x = conv2D(inputs, 64, kernel=(7,7), strides=(2,2))
x = tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=2, padding='same')(x)                         

### Block 1
x = meta_block(x, trips=[64, 64, 256], repeats=3, downsample=False, SE=True)

### Block 2
x = meta_block(x, trips=[128, 128, 512], repeats=4, SE=True)

### Block 3
x = meta_block(x, trips=[256, 256, 1024], repeats=6, SE=True)

### Block 4
x = meta_block(x, trips=[512, 512, 2048], repeats=3)

x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(num_classes, activation='softmax')(x) #logits 

SE_resnet = tf.keras.Model(inputs, x)

sgd = tf.keras.optimizers.SGD(lr=0.0, momentum=momentum, decay=0.0, nesterov=False) 

SE_resnet.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

class LossHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.lr = []
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.lr.append(step_decay(len(self.losses)))
        print('lr:', step_decay(len(self.losses)))

def step_decay(epoch):
    initial_lrate = initial_learning_rate
    drop = drop_rate
    epochs_drop = epochs_per_drop
    lrate = initial_lrate * math.pow(drop, math.floor((epoch)/epochs_drop))
    return lrate

# learning schedule callback
loss_history = LossHistory()
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
callbacks_list = [loss_history, lrate]

# fit the model
history = SE_resnet.fit(ds_train, steps_per_epoch=steps_per_epoch, validation_data=ds_test, epochs=epochs, callbacks=callbacks_list, validation_steps=100)

accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

np.savetxt("SE_resnet_cifar10_accuracy.csv", accuracy, delimiter=",")
np.savetxt("SE_resnet_cifar10_val_accuracy.csv", val_accuracy, delimiter=",")

Train for 1562 steps, validate for 100 steps
Epoch 1/100
1562/1562 [==============================] - 89s 57ms/step - loss: 2.1822 - accuracy: 0.3346 - val_loss: 1.6666 - val_accuracy: 0.4128
Epoch 2/100
1562/1562 [==============================] - 77s 49ms/step - loss: 1.6502 - accuracy: 0.4414 - val_loss: 1.7964 - val_accuracy: 0.3859
Epoch 3/100
1562/1562 [==============================] - 76s 49ms/step - loss: 1.4730 - accuracy: 0.4969 - val_loss: 1.5954 - val_accuracy: 0.4831
Epoch 4/100
1562/1562 [==============================] - 75s 48ms/step - loss: 1.3343 - accuracy: 0.5410 - val_loss: 1.5281 - val_accuracy: 0.4919
Epoch 5/100
1562/1562 [==============================] - 77s 49ms/step - loss: 1.2529 - accuracy: 0.5696 - val_loss: 1.7227 - val_accuracy: 0.4619
Epoch 6/100
1562/1562 [==============================] - 76s 49ms/step - loss: 1.1796 - accuracy: 0.5933 - val_loss: 1.4209 - val_accuracy: 0.5297
Epoch 7/100
1562/1562 [==============================] - 75s 48ms/step - 